## Imports

In [1]:
# General imports 
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import requests as r
from Bio import SeqIO
from io import StringIO
import warnings

warnings.filterwarnings('ignore')

# Import structuremap functions
import structuremap.utils
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score

structuremap.utils.set_logger()

## Set Parameters of Analysis

In [2]:
# Set parameters of analysis

analysis_threshold = 20 # number of amino acids either side to analyze

modifications = ["649.3660", "655.3735"] # which modifications we are looking for, as regex strings
heavy_modification = "655.3735"
light_modification = "649.3660"

## Load Dataset - MsrAKD

In [3]:
# Set correct pathing

curr_dir_path_str = "./"
curr_dir_path = os.path.abspath(curr_dir_path_str)

global_data_path_str = "../global_data"
global_data_path = os.path.abspath(global_data_path_str)

print("Current Directory: " + curr_dir_path)
print("Global Data Directory: " + global_data_path)

Current Directory: /Users/ritwiksrinivas/Desktop/Projects/ChULO_ABPP-playground/MsrKD
Global Data Directory: /Users/ritwiksrinivas/Desktop/Projects/ChULO_ABPP-playground/global_data


In [4]:
# Load initial dataset
data_loc = os.path.join(curr_dir_path, "05_10_24_293T_MsrKD_data.xlsx")
peptides = pd.read_excel(data_loc, sheet_name="293T_MsrAKD_quant")
peptides;

In [5]:
# Canonicalize data - none to do here
peptides;

In [ ]:
# Manual labeling of peptides
label_col_data = ["blue"] * 157 + ["green"] * 381 + ["white"] * 9 + ["red"] * 12 + ["gray"] * 104
label_col = pd.Series(label_col_data)
peptides["Color"] = label_col

#pd.set_option("display.max_rows", None)
#display(peptides)
#pd.reset_option("display.max_rows")
peptides;

In [ ]:
unique_uniprotIDs = peptides["Protein ID"].unique()
print("Unique UniProt IDs: \n" + str(unique_uniprotIDs))
print("Number of Unique UniProt IDs: " + str(unique_uniprotIDs.size))

In [ ]:
# Helper function to get full amino acid sequence for a protein
def get_complete_sequence(cID):
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,"fasta"))

    return str(pSeq[0].seq)

In [ ]:
# Load and update sequence cache df: mapping from UniProt IDs to complete AA sequence
# SLOW - ONLY DO THIS ONCE - CONVERT TO CSV FILE, THEN RE-LOAD FROM THAT FILE

path = os.path.join(global_data_path, "complete_sequence_cache.csv")
sequence_cache_df = pd.read_csv(path)
sequence_cache_df.set_index("Unnamed: 0", inplace=True)
sequence_cache_df.index.name = None
display(sequence_cache_df)

# Determine unknown sequences

unknown_uniprotIDs_idxs = ~np.isin(unique_uniprotIDs, sequence_cache_df["Protein ID"].values)
unknown_uniprotIDs = unique_uniprotIDs[unknown_uniprotIDs_idxs]
unknown_sequences_df = pd.DataFrame({"Protein ID": unknown_uniprotIDs})
display(unknown_sequences_df)

# Retrieve unknown sequences

tqdm.pandas()
unknown_sequences_df["Complete Sequence"] = unknown_sequences_df["Protein ID"].progress_apply(get_complete_sequence)
display(unknown_sequences_df)

sequence_cache_df_updated = pd.concat([sequence_cache_df, unknown_sequences_df])
sequence_cache_df_updated.to_csv(os.path.join(global_data_path, "complete_sequence_cache.csv"))
sequence_cache_df_updated;

In [ ]:
# Load cache df: mapping from UniProt IDs to complete AA sequence
path = os.path.join(global_data_path, "complete_sequence_cache.csv")
sequence_cache_df_updated = pd.read_csv(path)
sequence_cache_df_updated.set_index("Unnamed: 0", inplace=True)
sequence_cache_df_updated.index.name = None
sequence_cache_df_updated;

In [ ]:
peptides_cs = peptides.merge(sequence_cache_df_updated, how="left", on="Protein ID")
peptides_cs # cs means "complete sequence"

In [ ]:
# Create regex pattern to identify desired modifications
def create_modifications_pattern(modifications):

    whole, mantissa = modifications[0].split(".")
    pattern = r"M\[{}\.{}\]".format(whole, mantissa)

    for i in range(1, len(modifications)):
        whole, mantissa = modifications[i].split(".")
        pattern += r"|M\[{}\.{}\]".format(whole, mantissa)
    
    return pattern

modifications_pattern = create_modifications_pattern(modifications)
print(modifications_pattern)

In [ ]:
# extract left prefix of modified methionine (for indexing purposes)
IUPACCodes = "ACDEFGHIKLMNPQRSTVWY"
filtering = lambda string: ''.join([char for char in string if char in IUPACCodes])

peptides_completed_sequence["Peptide Sequence"] = peptides_completed_sequence["Light Modified Peptide"].map(filtering)
peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Sequence Location"] = pd.Series([a.find(b) for a, b in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Peptide Sequence"])])
peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Sequence Length"] = peptides_completed_sequence["Peptide Sequence"].str.len()
peptides_completed_sequence

In [ ]:
# sanity check - ensure sequence indexing is correct
temp = [A[B:B+C] for A, B, C in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Sequence Location"], peptides_completed_sequence["Sequence Length"])]
(temp == peptides_completed_sequence["Peptide Sequence"]).value_counts()

In [ ]:
# create regex pattern to identify desired modifications
left_prefix_pattern = "(.*)(" + modifications_pattern + ")"
print(left_prefix_pattern)

In [ ]:
# extract left prefix of modified methionine (for indexing purposes)
IUPACCodes = "ACDEFGHIKLMNPQRSTVWY"
filtering = lambda string: ''.join([char for char in string if char in IUPACCodes])

peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Light Modified Peptide"].str.extract(left_prefix_pattern)[0]
peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Left Prefix"].map(filtering)
peptides_completed_sequence["Left Prefix Length"] = peptides_completed_sequence["Left Prefix"].str.len()

peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Methionine Location"] = peptides_completed_sequence["Sequence Location"] + peptides_completed_sequence["Left Prefix Length"]
peptides_completed_sequence

In [ ]:
# Compute left/right analysis sequences based on threshold
peptides_completed_sequence[f"Left {analysis_threshold}"] = [A[B-analysis_threshold:B]  if (B - analysis_threshold >= 0) else A[0:B-1] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])]
peptides_completed_sequence[f"Right {analysis_threshold}"] = [A[B+1:B+1+analysis_threshold] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])]
peptides_completed_sequence

In [ ]:
pd.set_option('display.max_columns', None)
display(peptides_completed_sequence[~(peptides_completed_sequence["Site Number"] == peptides_completed_sequence["Methionine Location"] + 1)])
pd.reset_option('display.max_columns')

In [ ]:
# remove invalid proteins (according to alphafold) - TODO: attempt to incorporate these as well
# 7 invalid peptides as a result -> 2 blue, 4 green, 1 gray
invalid_IDs = ['Q09666', 'Q14204', 'Q9Y520', 'Q14789']
peptides_completed_sequence = peptides_completed_sequence[~peptides_completed_sequence["Protein ID"].isin(invalid_IDs)]
peptides_completed_sequence

# Download Alphafold Data - MsrAKD

In [ ]:
# path for alphafold protein data
alphafold_path_str = "../alphafold_data/"
alphafold_path = os.path.abspath(alphafold_path_str)

cif_dir = os.path.join(alphafold_path, "cif")
pae_dir = os.path.join(alphafold_path, "pae")

print(alphafold_path)
print(cif_dir)
print(pae_dir)

In [ ]:
# set uniprot IDs to use
uniprotIDs = peptides_completed_sequence["Protein ID"].unique()
uniprotIDs, len(uniprotIDs)

In [ ]:
# download cif data for proteins
# SLOW THE FIRST TIME
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=uniprotIDs,
    out_folder=cif_dir
)

In [ ]:
# download pae data for proteins
# SLOW THE FIRST TIME
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=uniprotIDs,
    out_folder=pae_dir, 
)

## Construct Alphafold Dataframe (Calculate Accessibilities) - MsrAKD

In [ ]:
# format alphafold data into dataframe
alphafold_annotation_MsrAKD = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=uniprotIDs)
alphafold_annotation_MsrAKD

In [ ]:
# calculate full sphere exposure -> radius = 2
exposure_sphere_rad2 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=2, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad2;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_annotation_MsrAKD.merge(
    exposure_sphere_rad2, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 3
exposure_sphere_rad3 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=3, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad3;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad3, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 4
exposure_sphere_rad4 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=4, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad4, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 4.5
exposure_sphere_rad4_5 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=4.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4_5;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad4_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 5
exposure_sphere_rad5 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 5.5
exposure_sphere_rad5_5 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=5.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5_5;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad5_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 6
exposure_sphere_rad6 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=6, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad6, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 6.5
exposure_sphere_rad6_5 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=6.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6_5;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad6_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 7
exposure_sphere_rad7 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=7, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad7, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 7.5
exposure_sphere_rad7_5 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=7.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7_5;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad7_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 8
exposure_sphere_rad8 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=8, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad8;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad8, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 12
exposure_sphere_rad12 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=12, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad12;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 18
exposure_sphere_rad18 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=18, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad18;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad18, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate full sphere exposure -> radius = 24
exposure_sphere_rad24 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad24;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_sphere_rad24, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD;

In [ ]:
# calculate part sphere exposure -> angle = 70, radius = 12
exposure_ang70_rad12 = annotate_accessibility(
    df=alphafold_annotation_MsrAKD, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)
exposure_ang70_rad12;

In [ ]:
alphafold_accessibility_MsrAKD = alphafold_accessibility_MsrAKD.merge(
    exposure_ang70_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrAKD

In [ ]:
alphafold_accessibility_MsrAKD.columns

In [ ]:
alphafold_accessibility_MsrAKD_smooth = get_smooth_score(
    alphafold_accessibility_MsrAKD, 
    np.array(['nAA_2_180_pae', 'nAA_3_180_pae', 'nAA_4_180_pae', 'nAA_4.5_180_pae', 'nAA_5_180_pae', 'nAA_5.5_180_pae', 'nAA_6_180_pae', 'nAA_6.5_180_pae', 'nAA_7_180_pae', 'nAA_7.5_180_pae', 'nAA_8_180_pae','nAA_12_180_pae', 'nAA_18_180_pae', 'nAA_24_180_pae', 'nAA_12_70_pae']), 
    [10])
alphafold_accessibility_MsrAKD_smooth;

In [ ]:
alphafold_accessibility_MsrAKD_smooth['IDR'] = np.where(
    alphafold_accessibility_MsrAKD_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)
alphafold_accessibility_MsrAKD_smooth

In [ ]:
alphafold_accessibility_MsrAKD_smooth.columns

# Merge Dataframes into Full Dataset (Includes Alphafold) - MsrAKD

In [ ]:
alphafold_accessibility_MsrAKD_smooth["position"] = alphafold_accessibility_MsrAKD_smooth["position"] - 1 # zero-index the positions to match initial dataframe

peptides_with_alphafold = peptides_completed_sequence.merge(
    alphafold_accessibility_MsrAKD_smooth, 
    how="left", 
    left_on=["Protein ID", "Methionine Location"], 
    right_on=["protein_id", "position"]
)
peptides_with_alphafold

In [ ]:
#peptides_with_alphafold.to_csv(os.path.join(curr_dir_path, "MsrAKD_with_alphafold.csv"))

In [ ]:
path = os.path.join(curr_dir_path, "MsrAKD_with_alphafold.csv")
peptides_with_alphafold = pd.read_csv(path)
peptides_with_alphafold.set_index("Unnamed: 0", inplace=True)
peptides_with_alphafold.index.name = None
peptides_with_alphafold

## Load Dataset - MsrB2KD

In [ ]:
data_loc = os.path.join(curr_dir_path, "05_10_24_293T_MsrKD_data.xlsx")
peptides = pd.read_excel(data_loc, sheet_name="293T_MsrB2KD_quant")
peptides

In [ ]:
# Canonicalize data - none to do here
peptides;

In [ ]:
label_col_data = ["blue"] * 10 + ["white"] * 30 + ["green"] * 381 + ["red"] * 213 + ["gray"] * 120
label_col = pd.Series(label_col_data)
peptides["color"] = label_col

#pd.set_option("display.max_rows", None)
display(peptides)
#pd.reset_option("display.max_rows")

In [ ]:
# helper function to get full amino acid sequence for a protein
def get_full_protein_seq(cID):
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,"fasta"))

    return str(pSeq[0].seq)

In [ ]:
# test - get a single amino acid sequence
#first_protein_ID = peptides["Protein ID"].iloc[0]
#test_sequence = get_full_protein_seq(first_protein_ID)
#print(test_sequence[575:587])
#print(peptides["Peptide Sequence"].iloc[0])

In [ ]:
unique_uniprotIDs = peptides["Protein ID"].unique()
unique_uniprotIDs, unique_uniprotIDs.size;

In [ ]:
# load known completed sequences
path = os.path.join(global_data_path, "uniprotID_to_complete_sequence_mapping.csv")
unique_IDs_to_sequence_df = pd.read_csv(path)
unique_IDs_to_sequence_df.set_index("Unnamed: 0", inplace=True)
unique_IDs_to_sequence_df.index.name = None
unique_IDs_to_sequence_df;

In [ ]:
unknown_uniprotIDs_idxs = ~np.isin(unique_uniprotIDs, unique_IDs_to_sequence_df["Protein ID"].values)
np.unique(unknown_uniprotIDs_idxs, return_counts=True)

In [ ]:
unknown_uniprotIDs = unique_uniprotIDs[unknown_uniprotIDs_idxs]
unknown_uniprotIDs, len(unknown_uniprotIDs);

In [ ]:
unknown_sequences_df = pd.DataFrame({"Protein ID": unknown_uniprotIDs})
unknown_sequences_df;

In [ ]:
# get whole amino acid sequences for unknown proteins
# SLOW - ONLY DO THIS ONCE - CONVERT TO CSV FILE, THEN RE-LOAD FROM THAT FILE

tqdm.pandas()
unknown_sequences_df["Complete Sequence"] = unknown_sequences_df["Protein ID"].progress_apply(get_full_protein_seq)
unknown_sequences_df

In [ ]:
unique_IDs_to_sequence_df_updated = pd.concat([unique_IDs_to_sequence_df, unknown_sequences_df])
unique_IDs_to_sequence_df_updated

In [ ]:
#unique_IDs_to_sequence_df_updated.to_csv(os.path.join(global_data_path, "uniprotID_to_complete_sequence_mapping.csv"))

In [ ]:
# load (updated) known completed sequences
path = os.path.join(global_data_path, "uniprotID_to_complete_sequence_mapping.csv")
unique_IDs_to_sequence_df = pd.read_csv(path)
unique_IDs_to_sequence_df.set_index("Unnamed: 0", inplace=True)
unique_IDs_to_sequence_df.index.name = None
unique_IDs_to_sequence_df

In [ ]:
peptides_completed_sequence = peptides.merge(unique_IDs_to_sequence_df, how="left", on="Protein ID")
peptides_completed_sequence.index = peptides.index
peptides_completed_sequence

In [ ]:
# create regex pattern to identify desired modifications
def create_modifications_pattern(modifications):

    split_mod = modifications[0].split(".")
    whole = split_mod[0]
    mantissa = split_mod[1]
    pattern = r"M\[{}\.{}\]".format(whole, mantissa)

    for i in range(1, len(modifications)):
        split_mod = modifications[i].split(".")
        whole = split_mod[0]
        mantissa = split_mod[1]
        pattern += r"|M\[{}\.{}\]".format(whole, mantissa)
    
    return pattern

modifications_pattern = create_modifications_pattern(modifications)
print(modifications_pattern)

In [ ]:
# extract left prefix of modified methionine (for indexing purposes)
IUPACCodes = "ACDEFGHIKLMNPQRSTVWY"
filtering = lambda string: ''.join([char for char in string if char in IUPACCodes])

peptides_completed_sequence["Peptide Sequence"] = peptides_completed_sequence["Light Modified Peptide"].map(filtering)
peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Sequence Location"] = pd.Series([a.find(b) for a, b in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Peptide Sequence"])])
peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Sequence Length"] = peptides_completed_sequence["Peptide Sequence"].str.len()
peptides_completed_sequence

In [ ]:
# sanity check - ensure sequence indexing is correct
temp = [A[B:B+C] for A, B, C in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Sequence Location"], peptides_completed_sequence["Sequence Length"])]
(temp == peptides_completed_sequence["Peptide Sequence"]).value_counts()

In [ ]:
# create regex pattern to identify desired modifications
left_prefix_pattern = "(.*)(" + modifications_pattern + ")"
print(left_prefix_pattern)

In [ ]:
# extract left prefix of modified methionine (for indexing purposes)
IUPACCodes = "ACDEFGHIKLMNPQRSTVWY"
filtering = lambda string: ''.join([char for char in string if char in IUPACCodes])

peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Light Modified Peptide"].str.extract(left_prefix_pattern)[0]
peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Left Prefix"].map(filtering)
peptides_completed_sequence["Left Prefix Length"] = peptides_completed_sequence["Left Prefix"].str.len()

peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Methionine Location"] = peptides_completed_sequence["Sequence Location"] + peptides_completed_sequence["Left Prefix Length"]
peptides_completed_sequence

In [ ]:
# Compute left/right analysis sequences based on threshold
peptides_completed_sequence[f"Left {analysis_threshold}"] = [A[B-analysis_threshold:B]  if (B - analysis_threshold >= 0) else A[0:B-1] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])]
peptides_completed_sequence[f"Right {analysis_threshold}"] = [A[B+1:B+1+analysis_threshold] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])]
peptides_completed_sequence

In [ ]:
pd.set_option('display.max_columns', None)
display(peptides_completed_sequence[~(peptides_completed_sequence["Site Number"] == peptides_completed_sequence["Methionine Location"] + 1)])
pd.reset_option('display.max_columns')

In [ ]:
# remove invalid proteins (according to alphafold) - TODO: attempt to incorporate these as well
# 12 invalid peptides as a result -> 5 green, 4 red, 3 gray
invalid_IDs = ['Q14204', 'Q09666', 'Q14789', 'Q9Y520', 'P46013', 'Q9NU22']
peptides_completed_sequence = peptides_completed_sequence[~peptides_completed_sequence["Protein ID"].isin(invalid_IDs)]
peptides_completed_sequence

# Download Alphafold Data - MsrB2KD

In [ ]:
# path for alphafold protein data
alphafold_path_str = "../alphafold_data/"
alphafold_path = os.path.abspath(alphafold_path_str)

cif_dir = os.path.join(alphafold_path, "cif")
pae_dir = os.path.join(alphafold_path, "pae")

print(alphafold_path)
print(cif_dir)
print(pae_dir)

In [ ]:
# set uniprot IDs to use
uniprotIDs = peptides_completed_sequence["Protein ID"].unique()
uniprotIDs, len(uniprotIDs)

In [ ]:
# download cif data for proteins
# SLOW THE FIRST TIME
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=uniprotIDs,
    out_folder=cif_dir
)

In [ ]:
# download pae data for proteins
# SLOW THE FIRST TIME
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=uniprotIDs,
    out_folder=pae_dir, 
)

## Construct Alphafold Dataframe (Calculate Accessibilities) - MsrB2KD

In [ ]:
# format alphafold data into dataframe
alphafold_annotation_MsrB2KD = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=uniprotIDs)
alphafold_annotation_MsrB2KD

In [ ]:
# calculate full sphere exposure -> radius = 2
exposure_sphere_rad2 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=2, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad2;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_annotation_MsrB2KD.merge(
    exposure_sphere_rad2, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 3
exposure_sphere_rad3 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=3, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad3;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad3, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 4
exposure_sphere_rad4 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=4, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad4, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 4.5
exposure_sphere_rad4_5 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=4.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad4_5;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad4_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 5
exposure_sphere_rad5 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 5.5
exposure_sphere_rad5_5 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=5.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad5_5;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad5_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 6
exposure_sphere_rad6 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=6, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad6, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 6.5
exposure_sphere_rad6_5 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=6.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad6_5;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad6_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 7
exposure_sphere_rad7 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=7, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad7, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 7.5
exposure_sphere_rad7_5 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=7.5, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad7_5;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad7_5, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 8
exposure_sphere_rad8 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=8, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad8;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad8, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 12
exposure_sphere_rad12 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=12, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad12;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 18
exposure_sphere_rad18 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=18, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad18;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad18, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate full sphere exposure -> radius = 24
exposure_sphere_rad24 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)
exposure_sphere_rad24;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_sphere_rad24, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD;

In [ ]:
# calculate part sphere exposure -> angle = 70, radius = 12
exposure_ang70_rad12 = annotate_accessibility(
    df=alphafold_annotation_MsrB2KD, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)
exposure_ang70_rad12;

In [ ]:
alphafold_accessibility_MsrB2KD = alphafold_accessibility_MsrB2KD.merge(
    exposure_ang70_rad12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_MsrB2KD

In [ ]:
alphafold_accessibility_MsrB2KD.columns

In [ ]:
alphafold_accessibility_MsrB2KD_smooth = get_smooth_score(
    alphafold_accessibility_MsrB2KD, 
    np.array(['nAA_2_180_pae', 'nAA_3_180_pae', 'nAA_4_180_pae', 'nAA_4.5_180_pae', 'nAA_5_180_pae', 'nAA_5.5_180_pae', 'nAA_6_180_pae', 'nAA_6.5_180_pae', 'nAA_7_180_pae', 'nAA_7.5_180_pae', 'nAA_8_180_pae','nAA_12_180_pae', 'nAA_18_180_pae', 'nAA_24_180_pae', 'nAA_12_70_pae']), 
    [10])
alphafold_accessibility_MsrB2KD_smooth;

In [ ]:
alphafold_accessibility_MsrB2KD_smooth['IDR'] = np.where(
    alphafold_accessibility_MsrB2KD_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)
alphafold_accessibility_MsrB2KD_smooth

In [ ]:
alphafold_accessibility_MsrB2KD_smooth.columns

# Merge Dataframes into Full Dataset (Includes Alphafold) - MsrB2KD

In [ ]:
alphafold_accessibility_MsrB2KD_smooth["position"] = alphafold_accessibility_MsrB2KD_smooth["position"] - 1 # zero-index the positions to match initial dataframe

peptides_with_alphafold = peptides_completed_sequence.merge(
    alphafold_accessibility_MsrB2KD_smooth, 
    how="left", 
    left_on=["Protein ID", "Methionine Location"], 
    right_on=["protein_id", "position"]
)
peptides_with_alphafold

In [ ]:
#peptides_with_alphafold.to_csv(os.path.join(curr_dir_path, "MsrB2KD_with_alphafold.csv"))

In [ ]:
path = os.path.join(curr_dir_path, "MsrB2KD_with_alphafold.csv")
peptides_with_alphafold = pd.read_csv(path)
peptides_with_alphafold.set_index("Unnamed: 0", inplace=True)
peptides_with_alphafold.index.name = None
peptides_with_alphafold

# The End (For Now)